In [1]:
import os
os.getcwd()

'C:\\Users\\T420\\Desktop\\UBIQUM\\5 PROJECT\\Quiniela\\notebooks'

In [2]:
import pandas as pd

xl = pd.ExcelFile('../bbdd/FULL_BBDD_with_features.xlsx')
df = xl.parse(xl.sheet_names[0])

In [18]:
df.columns

Index(['TEMPORADA', 'LIGA', 'JORNADA', 'FECHA', 'LOCAL', 'VISITANTE', 'GOL L',
       'GOL V', 'ARBITRO', 'QUINIELA', 'last2_hometeam_w', 'last2_hometeam_d',
       'last2_hometeam_l', 'last5_hometeam_w', 'last5_hometeam_d',
       'last5_hometeam_l', 'last10_hometeam_w', 'last10_hometeam_d',
       'last10_hometeam_l', 'last2_awayteam_w', 'last2_awayteam_d',
       'last2_awayteam_l', 'last5_awayteam_w', 'last5_awayteam_d',
       'last5_awayteam_l', 'last10_awayteam_w', 'last10_awayteam_d',
       'last10_awayteam_l', 'league_local_local_w', 'league_local_local_d',
       'league_local_local_l', 'league_visitor_visitor_w',
       'league_visitor_visitor_d', 'league_visitor_visitor_l', 'h2h_last_10_h',
       'h2h_last_10_d', 'h2h_last_10_a', 'h2h_5_homeaway_h',
       'h2h_5_homeaway_d', 'h2h_5_homeaway_a', 'ref_hometeam_w',
       'ref_hometeam_d', 'ref_hometeam_l', 'ref_awayteam_w', 'ref_awayteam_d',
       'ref_awayteam_l', 'ref_home_home_w', 'ref_home_home_d',
       'ref_home_ho

### Preparing models (create train and test, scaling, filtering information)

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

Select what part of the dataframe i want to model and create the final dataframe

In [46]:
jornada_cut = 10
league_selec = 1
selec = df[(df['JORNADA']>=jornada_cut) & (df['LIGA'] == league_selec)]

In [47]:
dummy1 = pd.get_dummies(selec.LOCAL)
dummy2 = pd.get_dummies(selec.VISITANTE)
quiniela = pd.get_dummies(selec.QUINIELA)
dummy2.columns = [i+"_2" for i in dummy2.columns]
selec = pd.concat([selec, dummy1, dummy2, quiniela], axis=1)

### Logistic Regression for "1"

In [48]:
x_col = list(selec.columns[[((i[-2:] in ['_l', '_w', '_d', '_h','_a' ]) | (i in dummy1.columns) |  
                             (i in dummy2.columns)) for i in list(selec.columns)]])
X = selec[x_col]
y = selec['1']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [50]:
from sklearn.metrics import make_scorer, accuracy_score, cohen_kappa_score, confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, y_train)
predslog_1 = clf.predict(X_test)
probslog_1 = clf.predict_proba(X_test)[:,1]
print('F1-Score:', round(f1_score(y_test, predslog_1),4), "|", "Kappa:", round(cohen_kappa_score(y_test, predslog_1),4))

F1-Score: 0.5537 | Kappa: 0.1808


### Logistic Regression for "2"

In [51]:
x_col = list(selec.columns[[((i[-2:] in ['_l', '_w', '_d', '_h','_a' ]) | (i in dummy1.columns) |  
                             (i in dummy2.columns)) for i in list(selec.columns)]])
X = selec[x_col]
y = selec['2']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [53]:
from sklearn.metrics import make_scorer, accuracy_score, cohen_kappa_score, confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs', max_iter=1000)
clf.fit(X_train, y_train)
predslog_2 = clf.predict(X_test)
probslog_2 = clf.predict_proba(X_test)[:,1]
print('F1-Score:', round(f1_score(y_test, predslog_2),4), "|", "Kappa:", round(cohen_kappa_score(y_test, predslog_2),4))

F1-Score: 0.2821 | Kappa: 0.1655


### Logistic Regression for "X"

In [54]:
x_col = list(selec.columns[[((i[-2:] in ['_l', '_w', '_d', '_h','_a' ]) | (i in dummy1.columns) |  
                             (i in dummy2.columns)) for i in list(selec.columns)]])
X = selec[x_col]
y = selec['X']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

In [56]:
from sklearn.metrics import make_scorer, accuracy_score, cohen_kappa_score, confusion_matrix, classification_report, f1_score
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='lbfgs')
clf.fit(X_train, y_train)
predslog_X = clf.predict(X_test)
probslog_X = clf.predict_proba(X_test)[:,1]
print('F1-Score:', round(f1_score(y_test, predslog_X),4), "|", "Kappa:", round(cohen_kappa_score(y_test, predslog_X),4))

F1-Score: 0.0045 | Kappa: -0.0023


### Paste Probabilities for 1X2 in new DF

In [81]:
import pandas as pd 
import numpy as np
zip_probs = list(zip(probslog_1, probslog_X, probslog_2)) 
probs_1X2 = pd.DataFrame(zip_probs, columns = ['predslog_1', 'predslog_X', 'predslog_2']) 

In [83]:
probs_1X2['1'] = probs_1X2.apply(lambda x: x[0]/(x[0]+x[1]+x[2]), axis=1)
probs_1X2['X'] = probs_1X2.apply(lambda x: x[1]/(x[0]+x[1]+x[2]), axis=1)
probs_1X2['2'] = probs_1X2.apply(lambda x: x[2]/(x[0]+x[1]+x[2]), axis=1)
probs_1X2.drop(["predslog_1", "predslog_X", "predslog_2"], axis=1, inplace=True)

In [85]:
probs_1X2.head()

,1,X,2
0,0.764888,0.106978,0.128133
1,0.427693,0.285559,0.286748
2,0.528488,0.184083,0.287430
3,0.648559,0.261369,0.090072
4,0.843640,0.075232,0.081128


In [ ]:
# df2.to_excel('prob x.xlsx', index=False)